# House Prices: Advanced Regression Techniques (Kaggle)

## 06-categoricals-in-the-neural-net

Sources:
* Kaggle competition: https://www.kaggle.com/c/house-prices-advanced-regression-techniques
* Check missing values (Will Koehrsen): https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction by Will Koehrsen
* Neural net implementation in PyTorch with embeddings (Yashu Seth): https://yashuseth.blog/2018/07/22/pytorch-neural-network-for-tabular-data-with-categorical-embeddings/ 
* Neural network embeddings explained (Will Koehrsen) https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526
* Sklearn pipelines: https://medium.com/dunder-data/from-pandas-to-scikit-learn-a-new-exciting-workflow-e88e2271ef62
* Pipelines with dataframes (John Ramey): https://ramhiser.com/post/2018-04-16-building-scikit-learn-pipeline-with-pandas-dataframe/


## Problem description

**Previous**:

**kaggle-houseprice-01-linear-model-and-continuous-imputation.ipynb**
We try to predict house prices based on a number of continuous and categorical variables.
In the first step, the prediction will be made using only a small selection of continuous variables:

* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* TotalBsmtSF: Total square feet of basement area
* SalePrice: target variable

We will use a very simple network: a linear network with a single non-linearity.

**kaggle-houseprice-02-data-scaling.ipynb**

In order to make it a little easier for gradient descent to converge to a minimum, we will scale the input data to have 0 mean and a standard deviation of 1. For a discussion on why it is useful to scale input data, see https://stats.stackexchange.com/questions/249378/is-scaling-data-0-1-necessary-when-batch-normalization-is-used. We will not scale the target data, following this discussion: https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re.

**kaggle-houseprice-03-one-hot-for-missing-continuous.ipynb**

Instead of just replacing missing values in our dataset with the mean or the median of the respective column, we will now create a *one-hot encoded vector* to mark the previously *missing data* and add it to the data set. For the same reason that we used the *sklearn.preprocessing StandardScaler* we will now make use of the *sklearn.impute Imputer* to replace missing values. Also, to make this part of data processing a little easier to reuse, we will refactor the code into a function. 

* missing_LotFrontage: one-hot vector with 1 for each missing value in LotFrontage and 0 else

**kaggle-houseprice-04-pipeline-for-preprocessing.ipynb**

Instead of relying on self-written code for processing our continuous variables we will now delegate this part of the processing to sklearn transformers. Additionally, those transformers will be put in a pipeline so that the transformers don't have to be called individually every time. This will help keeping our code simple and clean, and produce consistent results for processing multiple data.

* Add categorical variables
* Extend pipeline to handle categoricals
* Create a function to pre-process an arbitrary amount of dataframes at once

We still need to add more data to our model. In contrast to the first set of continuous variables, this time we will add categorical variables. Categorical variables differ from continuous variables in the fact that there may or may not be a natural order to values of a categorical variable, and that we cannot use categorical variables to do meaningful calculations (e.g. to calculate the mean, or a sum). For more information see https://en.wikipedia.org/wiki/Level_of_measurement.
Often those variables are represented by strings. In order to let our network handle categorical variables, we need to convert them to numbers (also called *factors* or *codes*). Additionally, we will expand our pre-processing pipeline to also handle missing values for categorical variables. We will also create a function that let's us use the pipeline on an arbitrary amount of dataframes at the same time.

New variables:
* MSZoning: Identifies the general zoning classification of the sale.
* MSSubClass: Identifies the type of dwelling involved in the sale.

**Now:**

* Using categorical variables in the neural net

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder

In [ ]:
# Show more rows and columns in the pandas output
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
#pd.set_option('display.width', 1000)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Helpers

In [ ]:
def show_missing(df, show_all=True):
    """    
    Shows absolute and relative number of missing values for each column of a dataframe,
    show_all=True also shows columns with no missing values.
    """
    mis_val_abs = df.isnull().sum()
    mis_val_rel = df.isnull().sum()/df.shape[0]
    mis_val_table = pd.concat([df.dtypes, mis_val_abs, mis_val_rel], axis=1)
    mis_val_table = mis_val_table.rename(columns={0: 'dtype', 1: 'Missing abs', 2: 'Missing rel'})

    if show_all:
        # Sort table descending by relative amount missing
        mis_val_table = mis_val_table.sort_values('Missing rel', ascending=False).round(3)
    else:
        # Sort table descending by relative amount missing, remove columns where no values are missing
        mis_val_table = mis_val_table[mis_val_table.iloc[:, 1] != 0].sort_values('Missing rel', ascending=False).round(3)
    
    return mis_val_table

In [ ]:
# The TypeSelector selects data from a dataframe based on its dtype. Credits to John Ramey, see sources on top.
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype=dtype
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])        

In [ ]:
def proc_df(cont_names, cat_names, *dataframes):
    """
    Pre-process arbitrary amount of dataframes with continuous and categorical variables.
    The respective fits are being calculated by combining all dataframes into a single
    dataframe.
    Returns one processed dataframe for each input dataframe.
    
    Parameters
    ----------
    cont_names : list
        List of column names for continuous variables.

    cat_names : list
        List of column names for categorical variables.

    *dataframes : pandas DataFrame(s)
        DataFrames to be processed.
    """
    
    df_combo = pd.DataFrame(columns=dataframes[0].columns)
    for arg in dataframes:
        df_combo = pd.concat([df_combo, arg], axis=0, sort=False)
        arg[cont_names] = arg[cont_names].astype('float64')
        arg[cat_names] = arg[cat_names].astype('category')
    
    # Convert columns in cont_names to *float64* dtype and the columns of cat_names to *category*.
    # This is necessary so that the TypeSelector in the pipeline can differentiate between cont and cat variables.
    # The pipeline can then apply different behaviour, according to the dtype.
    df_combo[cont_names] = df_combo[cont_names].astype('float64')
    df_combo[cat_names] = df_combo[cat_names].astype('category')
    
    # First, get names of columns with missing values.
    # The pipeline below then takes numeric features, in the order of appearance in the input dataframe.
    # The pipeline then takes categorical features in the order of appearance in the input dataframe.
    # All of these names are then merged to a list, and for the resulting dataframes.
    # This naming step is necessary because sklearn does not natively support pandas dataframes, and therefore
    #   all column names would be lost otherwise.
    missing_names = [f'mis_{name}' for name in df_combo.columns if df_combo[name].isnull().any()]
    ordered_cont_names = [col for col in df_combo.columns if col in cont_names]
    ordered_cat_names = [col for col in df_combo.columns if col in cat_names]
    names = missing_names + ordered_cont_names + ordered_cat_names
    
    preprocessing_pipeline = make_pipeline(
        FeatureUnion(transformer_list=[
            ('missing_features', make_pipeline(
                MissingIndicator(missing_values=np.nan)
            )),
            ('numeric_features', make_pipeline(
                TypeSelector('float64'),
                SimpleImputer(strategy='median'),
                StandardScaler()
            )),
            ('categorical_features', make_pipeline(
                TypeSelector('category'),
                SimpleImputer(strategy='most_frequent'),
                OrdinalEncoder()
            ))
        ])
    )
    preprocessing_pipeline.fit(df_combo)
        
    return (pd.DataFrame(preprocessing_pipeline.transform(arg), columns=names) for arg in dataframes)

## Load data

In [ ]:
PATH = Path('../data/houseprice/')
#!dir {PATH}  # For Windows
!ls {PATH}

In [ ]:
# Import training data
dep = ['SalePrice']
df_train = pd.read_csv(PATH/'train.csv', sep=',', header=0,
                       usecols=['MSZoning', 'MSSubClass', 'LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF',
                                'TotalBsmtSF', 'SalePrice'])
df_y = df_train[dep]
df_train = df_train.drop(dep, axis=1)
df_train.shape

In [ ]:
# Import test data
df_test = pd.read_csv(PATH/'test.csv', sep=',', header=0,
                       usecols=['MSZoning', 'MSSubClass', 'LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF',
                                'TotalBsmtSF'])

In [ ]:
# Define continuous and categorical columns
cat_names = ['MSZoning', 'MSSubClass']
cont_names = ['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF']

## Pre-processing

First, we take a look at a couple of rows and some descriptive statistics. This gives us an idea about the scale of values, and helps to decide if some continuous variables should perhaps be treated as categorical. In this case all variables will be treated as continuous.

We also check for missing values. If we find any, we have two options: remove the rows that contain missing values (which might lead to losing a lot of observations), or replace them with other values so that the network can use them. Common values used as a replacement are the mean or the median of the series, or some constant.

In [ ]:
df_train.head()

In [ ]:
df_train[cont_names].describe()

In [ ]:
# Categorical variables can be of type int or string. To show all cat columns in describe,
# we need to convert them to the same dtype
df_train[cat_names].astype('category').describe()

In [ ]:
pd.concat([show_missing(df_train), show_missing(df_test)], axis=1, sort=False)

In [ ]:
df_train, df_test = proc_df(cont_names, cat_names, df_train, df_test)

In [ ]:
df_train.head()

In [ ]:
# Redefine cat_names to include the new missing value columns
cat_names = [col for col in list(df_train.columns) if col not in cont_names]
cat_names

In [ ]:
pd.concat([show_missing(df_train), show_missing(df_test)], axis=1, sort=False)

In [ ]:
df_train.describe()

# PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

## Dataset, dataloader

In order to make the categorial variables distinct from the continuous variables when they're being used in the model, we will now split categorical and continuous variables for the training dataset.

Further, categorical variables will take the datatype torch.long, as opposed to torch.float32 for continuous variables.

In [ ]:
# Convert all data containers to tensors
t_train_cat = torch.tensor(df_train[cat_names].values, dtype=torch.long, device=device)
t_train_cont = torch.tensor(df_train[cont_names].values, dtype=torch.float32, device=device)
t_y = torch.tensor(df_y.values, dtype=torch.float32, device=device)

In [ ]:
# Dataset
train_ds = TensorDataset(t_train_cat, t_train_cont, t_y)

In [ ]:
# Dataloader
batch_size=64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

## Model

### New

The way to get the best out of categorical variables is to add them to the neural net by using embedding matrices.
The values in the categorical column vector are then just used as a lookup index of the respective embedding matrix.
The embedding matrices themselves will be initialized when the model is instantiated, just like any other parameter matrix, and updated during back propagation. This gives the neural net the chance to learn a richer representation about our categorical variables compared to just treating it like any other continuous variable.
For more discussion see the sources on top of the notebook.

In [ ]:
class LinearNet(nn.Module):
    def __init__(self, emb_dims, num_cont):
        super().__init__()        
        """
        Parameters
        ----------
        emb_dims : List of two element tuples
            This list will contain a two element tuple for each
            categorical feature. The first element of a tuple will
            denote the number of unique values of the categorical
            feature. The second element will denote the embedding
            dimension to be used for that feature.
        """
        
        self.num_cont = num_cont
        self.num_embeddings = sum([d for f, d in emb_dims])
        self.num_features = self.num_embeddings + self.num_cont
        
        # Embedding layers       
        if self.num_embeddings != 0:        
            self.emb_layers = nn.ModuleList(
                [nn.Embedding(f, d) for f, d in emb_dims]
            )            
        
        # Layers
        self.linear1 = nn.Linear(self.num_features, 100)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(100, 1)        
    
    def forward(self, x_cat, x_cont):
        if self.num_embeddings != 0:
            x = [emb_layer(x_cat[:, i]) for i, emb_layer in enumerate(self.emb_layers)]
            x = torch.cat(x, 1)
        
        if self.num_cont != 0:
            x = torch.cat([x, x_cont], 1)
        
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        
        return x

In [ ]:
def get_emb_dims(df):
    """
    Returns a list of tuples of the number of factors of a categorical
    variable and the minimum of half that number of factors + 1, and 50.
    
    Parameters
    ----------
    df : pandas.DataFrame
    """
    return [(df[name].nunique(), min(50, (df[name].nunique()+1)//2)) for name in df.columns]

### Embeddings step-by-step

This is based on the fastai impementation of embedding layers, see https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6

First, let's get the embedding dimensions for our embedding layers.

We determine the number of columns of each embedding layer based on the number of factors (that is, the number of unique values of a categorical variable) such that `num_cols = min( (num_factors+1)//2, 50)`.
This means that for the boolean column *Mis_MSZoning* (the result of the MissingIndicator above), the number of factors is 2 (since we have only 0 or 1 as possible values), and therefore the number of columns for this embedding layer is the minimum of (3//2, 50), which is 1.

In [ ]:
# If we're checking for the total number of factors, we need to consider all data
num_factors = pd.concat([df_train['mis_MSZoning'], df_test['mis_MSZoning']], axis=0).nunique()
num_cols = min((num_factors+1)//2, 50)
num_factors, num_cols

Let's check for MSZoning:

In [ ]:
num_factors = pd.concat([df_train['MSZoning'], df_test['MSZoning']], axis=0).nunique()
num_cols = min((num_factors+1)//2, 50)
num_factors, num_cols

So far, so good! Now we can do it for all categorical variables at once:

In [ ]:
emb_dims = get_emb_dims(pd.concat([df_train[cat_names], df_test[cat_names]], axis=0, sort=False))
emb_dims

The total number of columns over all embedding layers will give us the number of input values needed for categorical variables in the first linear layer of the network:

In [ ]:
num_embeddings = sum([y for x, y in emb_dims])
num_embeddings

Now let's manually define the embedding layers based on the dimensions we calculated before. PyTorch offers the `nn.Embedding` class to do that:

In [ ]:
emb_layer0 = nn.Embedding(2, 1)
emb_layer1 = nn.Embedding(2, 1)
emb_layer2 = nn.Embedding(2, 1)
emb_layer3 = nn.Embedding(16, 8)
emb_layer4 = nn.Embedding(5, 3)

Once instantiated, the embedding layers will also have random weights already initialized. For an alternative initialization, we can manually re-initialize the layers, but we will skip that for now:

In [ ]:
# torch.nn.init.kaiming_normal_(emb_layer.weight)
emb_layer0.weight, emb_layer1.weight, emb_layer2.weight, emb_layer3.weight, emb_layer4.weight

Finally, we can combine all embedding layers into a `nn.ModuleList`, which is basically a set of layers for the network.

In [ ]:
emb_layers = nn.ModuleList([emb_layer0, emb_layer1, emb_layer2, emb_layer3, emb_layer4])
emb_layers

The values of the categorical variables will be used as a lookup index for corresponding row in the respective embedding layer. Here is an example:

First, we convert the pandas dataframe into a torch.tensor. We check the size, which is 1460x5, which is the same as of the original dataframe with the categorical variables.

In [ ]:
x_cat = torch.tensor(df_train[cat_names].values, dtype=torch.long, device='cpu')
x_cat, x_cat.size()

The first row contains the values [0, 0, 0, 5, 3].
We will now get the rows

* 0 from emb_layer0
* 0 from emb_layer1
* 0 from emb_layer2
* 5 from emb_layer3
* 3 from emb_layer4

and concatenate them to a vector.

In [ ]:
# x = [emb_layer(cat_data[:, i]) for i,emb_layer in enumerate(self.emb_layers)]

In [ ]:
x = [emb_layer(x_cat[0, i]) for i,emb_layer in enumerate(emb_layers)]
x

In [ ]:
torch.cat(x)

As we can see, we now have 14 values. 1 from emb_layer0, 1 from emb_layer1, 1 from the emb_layer2, 8 from emb_layer3 and 5 from emb_layer4. The values correspond to the weigths in the respective embedding layer.

Of course we can also do that for all rows at once:

In [ ]:
x = [emb_layer(x_cat[:, i]) for i, emb_layer in enumerate(emb_layers)]
x = torch.cat(x, 1)
x.size()

In the neural network, this tensor will be concatenated to the tensor that holds the continuous variables, and will serve as input matrix for the first linear layer.

### Model instance

In [ ]:
# Instantiate the model
model = LinearNet(emb_dims=emb_dims, num_cont=len(cont_names)).to(device)

## Optimizer

In [ ]:
# Learning rate and optimizer
lr = 0.1
opt = torch.optim.Adam(model.parameters(), lr=lr)

## Loss

In [ ]:
loss_fn = F.mse_loss

## Train

In [ ]:
losses = []
def fit(num_epochs, model, loss_fn, opt):    
    for epoch in range(num_epochs):
        for xb_cat, xb_cont, yb in train_dl:
            # Forward
            #xb_cat, xb_cont, yb = xb_cat.to(device), xb_cont.to(device), yb.to(device)
            preds = model(xb_cat, xb_cont)
            loss = loss_fn(preds, yb)
            losses.append(loss)
            
            # Gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        if epoch%20==0:
            print('Training loss:', loss_fn(model(t_train_cat, t_train_cont), t_y))

In [ ]:
# Train for 300 epochs
fit(num_epochs=300, model=model, loss_fn=loss_fn, opt=opt)

In [ ]:
plt.plot(losses)

In [ ]:
preds = model(t_train_cat, t_train_cont)

In [ ]:
torch.cat([preds, t_y.reshape(-1,1)], dim=1)[:10, :]

Adding categorical variables and embeddings has helped us to to decrease the loss further, by approximately 20% (2.2 vs 1.8).

In [ ]:
plt.scatter(preds.detach().cpu().numpy(), t_y.reshape(-1,1).detach().cpu())